In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from scipy import sparse
from sklearn.svm import SVC
import gc
gc.enable()

Supervised NLP requires a pre-labelled dataset for training and testing, and is generally interested in categorizing text in various ways. In this case, we are going to try to predict whether a sentence comes from _Alice in Wonderland_ by Lewis Carroll or _Persuasion_ by Jane Austen. We can use any of the supervised models we've covered previously, as long as they allow categorical outcomes. In this case, we'll try Random Forests, SVM, and KNN.

Our feature-generation approach will be something called _BoW_, or _Bag of Words_. BoW is quite simple: For each sentence, we count how many times each word appears. We will then use those counts as features.  

In [2]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# The Chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)
    
alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)

In [3]:
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [4]:
# Group into sentences.
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !, I, shall, be, late, !, ')",Carroll
4,"((, when, she, thought, it, over, afterwards, ...",Carroll


Time to bag some words!  Since spaCy has already tokenized and labelled our data, we can move directly to recording how often various words occur.  We will exclude stopwords and punctuation.  In addition, in an attempt to keep our feature space from exploding, we will work with lemmas (root words) rather than the raw text terms, and we'll only use the 2000 most common words for each text.

In [5]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [6]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500


,hedge,rock,bind,desert,humoured,beau,savage,chief,pepper,ahem,...,lively,power,rapid,elliots,despair,sore,lazily,june,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !, I, shall, be, late, !, ')",Carroll
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"((, when, she, thought, it, over, afterwards, ...",Carroll


## Trying out BoW

Now let's give the bag of words features a whirl by trying a random forest.

In [7]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split

rfc = ensemble.RandomForestClassifier()
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

Training set score: 0.990088105727

Test set score: 0.929515418502


Holy overfitting, Batman! Overfitting is a known problem when using bag of words, since it basically involves throwing a massive number of features at a model – some of those features (in this case, word frequencies) will capture noise in the training set. Since overfitting is also a known problem with Random Forests, the divergence between training score and test score is expected.


## BoW with Logistic Regression

Let's try a technique with some protection against overfitting due to extraneous features – logistic regression with lasso.

In [8]:
from sklearn.linear_model import LogisticRegression, ElasticNet

lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(2724, 3023) (2724,)
Training set score: 0.963656387665

Test set score: 0.935572687225


Logistic regression performs a bit better than the random forest.  

# BoW with Gradient Boosting

And finally, let's see what gradient boosting can do:

In [9]:
clf = ensemble.GradientBoostingClassifier(verbose=True)
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

      Iter       Train Loss   Remaining Time 
         1           1.0642           27.14s
         2           1.0133           27.09s
         3           0.9749           26.80s
         4           0.9402           26.87s
         5           0.9123           27.14s
         6           0.8877           27.92s
         7           0.8673           27.28s
         8           0.8490           26.73s
         9           0.8328           26.32s
        10           0.8186           26.55s
        20           0.7222           24.70s
        30           0.6672           21.62s
        40           0.6251           18.44s
        50           0.5947           15.41s
        60           0.5698           12.27s
        70           0.5458            9.18s
        80           0.5279            6.09s
        90           0.5093            3.03s
       100           0.4936            0.00s
Training set score: 0.92107195301

Test set score: 0.90859030837


Looks like logistic regression is the winner, but there's room for improvement.

# Same model, new inputs

What if we feed the model a different novel by Jane Austen, like _Emma_?  Will it be able to distinguish Austen from Carroll with the same level of accuracy if we insert a different sample of Austen's writing?

First, we need to process _Emma_ the same way we processed the other data, and combine it with the Alice data:

In [10]:
# Clean the Emma data.
emma = gutenberg.raw('austen-emma.txt')
emma = re.sub(r'VOLUME \w+', '', emma)
emma = re.sub(r'CHAPTER \w+', '', emma)
emma = text_cleaner(emma)
print(emma[:100])

Emma Woodhouse, handsome, clever, and rich, with a comfortable home and happy disposition, seemed to


In [11]:
# Parse our cleaned data.
emma_doc = nlp(emma)

In [12]:
# Group into sentences.
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]
emma_sents = [[sent, "Austen"] for sent in emma_doc.sents]

# Emma is quite long, let's cut it down to the same length as Alice.
emma_sents = emma_sents[0:len(alice_sents)]

In [13]:
# Build a new Bag of Words data frame for Emma word counts.
# We'll use the same common words from Alice and Persuasion.
emma_sentences = pd.DataFrame(emma_sents)
emma_bow = bow_features(emma_sentences, common_words)

print('done')

Processing row 0
Processing row 500
Processing row 1000
done


In [14]:
# Now we can model it!
# Let's use logistic regression again.

# Combine the Emma sentence data with the Alice data from the test set.
X_Emma_test = np.concatenate((
    X_train[y_train[y_train=='Carroll'].index],
    emma_bow.drop(['text_sentence','text_source'], 1)
), axis=0)
y_Emma_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Austen'] * emma_bow.shape[0])])

# Model.
print('\nTest set score:', lr.score(X_Emma_test, y_Emma_test))
lr_Emma_predicted = lr.predict(X_Emma_test)
pd.crosstab(y_Emma_test, lr_Emma_predicted)


Test set score: 0.703466666667


col_0,Austen,Carroll
row_0,,
Austen,1158,21
Carroll,535,161


Well look at that!  NLP approaches are generally effective on the same type of material as they were trained on. It looks like this model is actually able to differentiate multiple works by Austen from Alice in Wonderland.  Now the question is whether the model is very good at identifying Austen, or very good at identifying Alice in Wonderland, or both...

# Challenge 0:

Recall that the logistic regression model's best performance on the test set was 93%.  See what you can do to improve performance.  Suggested avenues of investigation include: Other modeling techniques (SVM?), making more features that take advantage of the spaCy information (include grammar, phrases, POS, etc), making sentence-level features (number of words, amount of punctuation), or including contextual information (length of previous and next sentences, words repeated from one sentence to the next, etc), and anything else your heart desires.  Make sure to design your models on the test set, or use cross_validation with multiple folds, and see if you can get accuracy above 97%.  

In [15]:
car_df = word_counts.drop(['text_sentence', 'text_source'], axis=1)[word_counts.text_source=='Carroll'].sum(axis=0)
aus_df = word_counts.drop(['text_sentence', 'text_source'], axis=1)[word_counts.text_source=='Austen'].sum(axis=0)
dfs = pd.DataFrame(columns=car_df.index, index=['Carroll', 'Austen'])
dfs.loc['Carroll'] = car_df
dfs.loc['Austen'] = aus_df
dfs.loc['tot'] = dfs.iloc[:2].sum(axis=0)
dfs.loc['c_part'] = dfs.loc['Carroll']/dfs.loc['tot']
dfs.loc['a_part'] = dfs.loc['Austen']/dfs.loc['tot']
class_imb = len(word_counts[word_counts.text_source=='Austen'])/len(word_counts)
dfs.loc['c_part'] = dfs.loc['c_part'] * (class_imb)
dfs.loc['a_part'] = dfs.loc['a_part'] * (1-class_imb)
dfs.loc['min_part'] = dfs.iloc[-2:].min(axis=0)
dfs.head(6)

,hedge,rock,bind,desert,humoured,beau,savage,chief,pepper,ahem,...,truly,desk,lively,power,rapid,elliots,despair,sore,lazily,june
Carroll,3.000000,1.000000,1.000000,0.000000,0.000000,4.000000,4.000000,1.000000,7.000000,1.000000,...,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000
Austen,7.000000,3.000000,3.000000,4.000000,3.000000,0.000000,0.000000,3.000000,0.000000,0.000000,...,12.000000,0.000000,6.000000,18.000000,4.000000,6.000000,1.000000,3.000000,0.000000,3.000000
tot,10.000000,4.000000,4.000000,4.000000,3.000000,4.000000,4.000000,4.000000,7.000000,1.000000,...,12.000000,2.000000,6.000000,18.000000,4.000000,6.000000,2.000000,3.000000,1.000000,3.000000
c_part,0.222093,0.185077,0.185077,0.000000,0.000000,0.740308,0.740308,0.185077,0.740308,0.740308,...,0.000000,0.740308,0.000000,0.000000,0.000000,0.000000,0.370154,0.000000,0.740308,0.000000
a_part,0.181784,0.194769,0.194769,0.259692,0.259692,0.000000,0.000000,0.194769,0.000000,0.000000,...,0.259692,0.000000,0.259692,0.259692,0.259692,0.259692,0.129846,0.259692,0.000000,0.259692
min_part,0.181784,0.185077,0.185077,0.000000,0.000000,0.000000,0.000000,0.185077,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.129846,0.000000,0.000000,0.000000


In [220]:
filtered_words = np.where((dfs.loc['min_part'].values<.07), dfs.loc['min_part'].index, 'NaN')
word_counts['length_'] = word_counts.text_sentence.apply(len)
word_counts['has_!'] = word_counts.text_sentence.apply(lambda x: '!' in x.orth_)
word_counts['has_num'] = word_counts.text_sentence.apply(lambda z: any([x.isnumeric() for x in z.orth_]))
word_counts['has_alic'] = word_counts.text_sentence.apply(lambda x: ('Alice' in x.orth_))

def get_mwl(sentence):
    sentence = sentence.orth_
    return pd.Series(sentence.split(' ')).apply(len).mean()
    
word_counts['mwl'] = word_counts.text_sentence.apply(get_mwl)
word_counts['long_words'] = np.where(word_counts['mwl']>4.875, True, False)

def get_longest_word(sentence):
    sentence = sentence.orth_
    return pd.Series(sentence.split(' ')).apply(len).max()
    
word_counts['longest_word'] = word_counts.text_sentence.apply(get_longest_word)

def find_has_caps(sent):
    for x in sent:
        if len(x.orth_) > 1 and x.orth_.isupper():
            return True
    return False

word_counts['has_all_caps'] = word_counts.text_sentence.apply(find_has_caps)

filtered_words = list(set(filtered_words))
filtered_words.remove('NaN')
X = word_counts[filtered_words + ['has_all_caps']]
print(X.shape)

(4540, 2194)


## Best Result: 96.7% W/ Naive Bayes

In [244]:
gc.collect()
X = word_counts[filtered_words + ['has_all_caps']]
mod = BernoulliNB()
params = {
    'alpha': [.5]
}

srch = GridSearchCV(mod, params, cv=10, verbose=0, n_jobs=1)
srch.fit(X, Y)
print(srch.best_score_, srch.best_params_)

0.96718061674 {'alpha': 0.5}


# Challenge 1:
Find out whether your new model is good at identifying Alice in Wonderland vs any other work, Persuasion vs any other work, or Austen vs any other work.  This will involve pulling a new book from the Project Gutenberg corpus (print(gutenberg.fileids()) for a list) and processing it.

Record your work for each challenge in a notebook and submit it below.

In [245]:
model = BernoulliNB(alpha=.5)
model.fit(X, Y)

BernoulliNB(alpha=0.5, binarize=0.0, class_prior=None, fit_prior=True)

## Other Work and Notes Below

In [86]:
gc.collect()
X = word_counts[['longest_word', 'mwl', 'has_!', 'has_alic']]
mod = BernoulliNB()
params = {
    'alpha': [.1, .3, .5, .7]
}

srch = GridSearchCV(mod, params, cv=10, verbose=1, n_jobs=1)
srch.fit(X, Y)
print(srch.best_score_, srch.best_params_)

Fitting 10 folds for each of 4 candidates, totalling 40 fits
0.820704845815 {'alpha': 0.1}


[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    1.3s finished


In [134]:
Yc = np.where(Y=='Carroll', 1, 0)
Yc.sum()/len(Y)

0.25969162995594713

In [223]:
gc.collect()
X = word_counts
model_scores = []
ensemble_scores = []
got_wrong = []
splt = StratifiedKFold(n_splits=10)
for itrain, itest in splt.split(X, Y):
    mod1 = BernoulliNB(alpha=.5)
    mod1.fit(X[filtered_words + ['has_all_caps']].loc[itrain], Y.loc[itrain])
    probs = mod1.predict_proba(X[filtered_words+ ['has_all_caps']].loc[itest])
    preds1 = mod1.predict(X[filtered_words+ ['has_all_caps']].loc[itest])
    high_conf = np.where(probs.max(axis=1)>.8, True, False)
    low_conf = np.where(high_conf, False, True)
    acc = np.where(preds1 == Y.loc[itest], 1, 0).sum()/len(Y.loc[itest])
    #print('model 1 accuracy: ', acc)
    model_scores.append(acc)
    #print('accuracy at confidence threshold: ', np.where((high_conf) & (preds1==Y.loc[itest]), 1, 0).sum()/len(Y[high_conf]))
    #print('proportion of predictions: ', (len(Y[high_conf])/len(Y.loc[itest])))

    #mod2 = MultinomialNB(alpha=.1)

    #mod2.fit(X[['longest_word', 'mwl', 'has_!', 'has_alic']].loc[itrain], Y.loc[itrain])
    #preds2 = mod2.predict(X[['longest_word', 'mwl', 'has_!', 'has_alic']].loc[itest])
    #master_preds = preds1
    master_preds = np.where(low_conf & (X.loc[itest]['has_!']), 'Austen', preds1)
    ens_score = np.where(master_preds==Y.loc[itest], 1, 0).sum()/len(Y.loc[itest])
    #print('ensemble score: ', ens_score)
    ensemble_scores.append(ens_score)

    got_wrong += list(np.where((preds1 != Y.loc[itest]), itest, 0))
    
    
print('-'*50, '\n')
print('mean model 1 score: ', pd.Series(model_scores).mean())
print('mean ensemble score: ', pd.Series(ensemble_scores).mean())
got_wrong=list(set(got_wrong))
got_wrong.remove(0)

-------------------------------------------------- 

mean model 1 score:  0.96718300303
mean ensemble score:  0.964101723077


In [226]:
len(mislabeled)

149

In [225]:
mislabeled = word_counts.loc[got_wrong]
len(mislabeled[(mislabeled.text_source=='Austen')])/len(mislabeled)

0.18120805369127516

In [214]:
#[x.orth_.isupper() for x in mislabeled.iloc[11].text_sentence]

def find_has_caps(sent):
    for x in sent:
        if len(x.orth_) > 1 and x.orth_.isupper():
            return True
    return False

find_has_caps(mislabeled.iloc[12].text_sentence)

False

In [240]:
mislabeled.iloc[12].text_source

'Carroll'

In [46]:
gc.collect()
linreg = LogisticRegression()
X = word_counts.drop(['text_source', 'text_sentence'], axis=1)
params = {
    'penalty': ['l1', 'l2'],
    'C': [.1, .5, 1, 5, 10]
}

srch = GridSearchCV(linreg, params, cv=10, verbose=10, n_jobs=1)
srch.fit(X, Y)
print(srch.best_score_, srch.best_params_)

Fitting 10 folds for each of 10 candidates, totalling 100 fits
[CV] C=0.1, penalty=l1 ...............................................
[CV] ...... C=0.1, penalty=l1, score=0.8703296703296703, total=   1.0s
[CV] C=0.1, penalty=l1 ...............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.7s remaining:    0.0s


[CV] ...... C=0.1, penalty=l1, score=0.8502202643171806, total=   0.9s
[CV] C=0.1, penalty=l1 ...............................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    3.3s remaining:    0.0s


[CV] ...... C=0.1, penalty=l1, score=0.8568281938325991, total=   0.9s
[CV] C=0.1, penalty=l1 ...............................................


...

[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   14.9s remaining:    0.0s


[CV] ....... C=0.1, penalty=l1, score=0.869757174392936, total=   1.0s
[CV] C=0.1, penalty=l2 ...............................................
[CV] ...... C=0.1, penalty=l2, score=0.8857142857142857, total=   1.0s
[CV] C=0.1, penalty=l2 ...............................................
[CV] ...... C=0.1, penalty=l2, score=0.8744493392070485, total=   1.0s
[CV] C=0.1, penalty=l2 ...............................................
[CV] ...... C=0.1, penalty=l2, score=0.8700440528634361, total=   1.1s
[CV] C=0.1, penalty=l2 ...............................................
[CV] ...... C=0.1, penalty=l2, score=0.8766519823788547, total=   1.1s
[CV] C=0.1, penalty=l2 ...............................................
[CV] ...... C=0.1, penalty=l2, score=0.9185022026431718, total=   0.9s
[CV] C=0.1, penalty=l2 ...............................................
[CV] ...... C=0.1, penalty=l2, score=0.9273127753303965, total=   1.0s
[CV] C=0.1, penalty=l2 ...............................................
[CV] .

[CV] ........ C=5, penalty=l1, score=0.9361233480176211, total=   1.0s
[CV] C=5, penalty=l1 .................................................
[CV] ........ C=5, penalty=l1, score=0.9162995594713657, total=   1.0s
[CV] C=5, penalty=l1 .................................................
[CV] ........ C=5, penalty=l1, score=0.9403973509933775, total=   1.0s
[CV] C=5, penalty=l2 .................................................
[CV] ........ C=5, penalty=l2, score=0.9428571428571428, total=   1.0s
[CV] C=5, penalty=l2 .................................................
[CV] ........ C=5, penalty=l2, score=0.9140969162995595, total=   1.0s
[CV] C=5, penalty=l2 .................................................
[CV] ........ C=5, penalty=l2, score=0.8986784140969163, total=   1.0s
[CV] C=5, penalty=l2 .................................................
[CV] ........ C=5, penalty=l2, score=0.9118942731277533, total=   1.0s
[CV] C=5, penalty=l2 .................................................
[CV] .

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  2.7min finished


0.929515418502 {'C': 10, 'penalty': 'l2'}


In [169]:
gc.collect()
splt = StratifiedKFold(n_splits=5)
for itrain, itest in splt.split(X, Y):
    mod1 = BernoulliNB(alpha=.5)
    mod1.fit(X.loc[itrain], Y.loc[itrain])
    probs = mod1.predict_proba(X.loc[itest])
    preds1 = mod1.predict(X.loc[itest])
    high_conf = np.where(probs.max(axis=1)>.55, True, False)
    low_conf = np.where(high_conf, False, True)
    print('accuracy at confidence threshold: ', np.where((high_conf) & (preds1==Y.loc[itest]), 1, 0).sum()/len(Y[high_conf]))
    print('proportion of predictions: ', (len(Y[high_conf])/len(Y.loc[itest])))

    mod2 = ensemble.GradientBoostingClassifier(
        verbose=False, 
        n_estimators=6000,
        learning_rate=.05,
        subsample=.18,
        max_features=15
    )
    mod2.fit(X.loc[itrain], Y.loc[itrain])
    preds2 = mod2.predict(X.loc[itest])
    master_preds = np.where(high_conf, preds1, preds2)
    ens_score = np.where(master_preds==Y.loc[itest], 1, 0).sum()/len(Y.loc[itest])
    print('ensemble score: ', ens_score)

/home/alliedtoasters/anaconda3/envs/dspy3/lib/python3.6/site-packages/pandas/core/internals.py:258: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 4540 but corresponding boolean dimension is 909
  return self.values[slicer]
/home/alliedtoasters/anaconda3/envs/dspy3/lib/python3.6/site-packages/pandas/core/indexes/base.py:1700: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 4540 but corresponding boolean dimension is 909
  result = getitem(key)


accuracy at confidence threshold:  0.965440356745
proportion of predictions:  0.9867986798679867
ensemble score:  0.95599559956


/home/alliedtoasters/anaconda3/envs/dspy3/lib/python3.6/site-packages/pandas/core/internals.py:258: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 4540 but corresponding boolean dimension is 908
  return self.values[slicer]
/home/alliedtoasters/anaconda3/envs/dspy3/lib/python3.6/site-packages/pandas/core/indexes/base.py:1700: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 4540 but corresponding boolean dimension is 908
  result = getitem(key)


accuracy at confidence threshold:  0.954796030871
proportion of predictions:  0.998898678414097
ensemble score:  0.954845814978
accuracy at confidence threshold:  0.973539140022
proportion of predictions:  0.998898678414097


KeyboardInterrupt: 

In [63]:
np.asarray(X)
X = sparse.coo_matrix(X)
    

In [82]:
np.sqrt(X.shape[1])

46.840153714521477

In [108]:
gc.collect()
clf = ensemble.GradientBoostingClassifier(verbose=True, class_weight='balanced')
params = {
    'n_estimators': [6000],
    'learning_rate': [.05],
    'subsample': [.18],
    'max_features': [15],
}

srch = GridSearchCV(clf, params, cv=5, verbose=10, n_jobs=1)
srch.fit(X, Y)
print(srch.best_score_, srch.best_params_)

TypeError: __init__() got an unexpected keyword argument 'class_weight'

In [43]:
pd.Series(word_counts[word_counts.text_source=='Carroll'].text_sentence.iloc[45].orth_.split(' ')).apply(len).mean()

4.25

In [73]:
word_counts[word_counts.text_source=='Austen'].text_sentence.iloc[8]

Few women could think more of their personal appearance than he did, nor could the valet of any new made lord be more delighted with the place he held in society.

In [92]:
re.match(r'[A-Z][A-Z]+', 'AL')

In [117]:
word_counts.columns

Index(['eldest', 'italian', 'merrily', 'middle', 'rap', 'connect', 'delicate',
       'intend', 'point', 'expression',
       ...
       'kindness', 'ful', 'surprised', 'indisposition', 'essential', 'utility',
       'cupboard', 'text_sentence', 'text_source', 'length_'],
      dtype='object', length=3026)

In [ ]:
X = sparse.coo_matrix(X)

svc = SVC(class_weight='balanced')
params = {
    'C': [.1, 1, 10],
    'gamma': [.1, 1]
}

srch = GridSearchCV(svc, params, cv=10, verbose=2, n_jobs=2)
srch.fit(X, Y)
print(srch.best_score_, srch.best_params_)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(np.asarray(X), 
                                                    np.asarray(Y),
                                                    test_size=0.4,
                                                    random_state=0)

Y_train = np.where(y_train=='Austen', 1, 0)
Y_test = np.where(y_test=='Austen', 1, 0)

dtrain = xgb.DMatrix(X_train, label=Y_train)
dtest = xgb.DMatrix(X_test, label=Y_test)

In [ ]:
class_imb = len(word_counts[word_counts.text_source=='Austen'])/len(word_counts)

In [ ]:
param = {
    'max_depth': 6, 
    'eta': .05,
    'silent': 0, 
    'objective': 
    'binary:logistic', 
    'scale_pos_weight': (class_imb),  #Tell model about class imbalance
    'nthread':4,
    'eval_metric':'error'
}

plst = param.items()
num_round = 500
evallist = [(dtest, 'eval'), (dtrain, 'train')]
bst = xgb.train(plst, dtrain, num_round, evallist, verbose_eval=True) #We can watch outputs to see model slowly optimize.
#Breaks 90% accuracy at around 410 iterations, with vanishing returns around 500.

Y_ = np.where(bst.predict(dtrain)>.5, 1, 0)
tb = pd.crosstab(Y_, Y_train)
print(tb)
print((tb.iloc[0, 0]+tb.iloc[1, 1])/tb.sum().sum())

Y_ = np.where(bst.predict(dtest)>.5, 1, 0)
tb = pd.crosstab(Y_, Y_test)
print(tb)
print((tb.iloc[0, 0]+tb.iloc[1, 1])/tb.sum().sum())

In [ ]:
unique_pos = []
for sentence in word_counts['text_sentence']:
    for token in sentence:
        unique_pos.append(token.pos_)
        
unique_pos = list(set(unique_pos))

def pos_features(sentences, unique_pos):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns = unique_pos, index=pd.Series(sentences).index)
    df.loc[:, unique_pos] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(sentences):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        poss = [token.pos_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                 )]
        
        # Populate the row with word counts.
        for pos in poss:
            df.loc[i, pos] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df

pos_counts = pos_features(word_counts.text_sentence, unique_pos)
pos_counts = pos_counts.drop(['CCONJ'], axis=1)

In [ ]:
pos_densities = []
for pos in pos_counts.columns:
    pos_counts['{}_density'.format(pos)] = pos_counts[pos]/word_counts['length_']
    pos_densities.append('{}_density'.format(pos))

In [53]:
X = np.asarray(pd.concat([word_counts[filtered_words], pos_counts[pos_densities]], axis=1))
mod = MultinomialNB()
params = {
    'alpha': [.8, .9]
}

srch = GridSearchCV(mod, params, cv=5, verbose=1, n_jobs=2)
srch.fit(X, Y)
print(srch.best_score_, srch.best_params_)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=2)]: Done  10 out of  10 | elapsed:   17.5s finished


0.957929515419 {'alpha': 0.8}


In [56]:
last_deps = []
for sent in word_counts.text_sentence:
    last_deps.append(sent[0].dep_)
    
last_deps = pd.Series(last_deps)

In [57]:
first_deps = []
for sent in word_counts.text_sentence:
    first_deps.append(sent[0].dep_)
    
first_deps = pd.Series(first_deps)
X = pd.concat([pd.get_dummies(first_deps), word_counts[filtered_words]], axis=1)
mod = BernoulliNB()
params = {
    'alpha': [1]
}

srch = GridSearchCV(mod, params, cv=5, verbose=2)
srch.fit(X, Y)
print(srch.best_score_, srch.best_params_)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] alpha=1 .........................................................
[CV] .......................................... alpha=1, total=   1.0s
[CV] alpha=1 .........................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.7s remaining:    0.0s


[CV] .......................................... alpha=1, total=   1.0s
[CV] alpha=1 .........................................................
[CV] .......................................... alpha=1, total=   0.9s
[CV] alpha=1 .........................................................
[CV] .......................................... alpha=1, total=   1.0s
[CV] alpha=1 .........................................................
[CV] .......................................... alpha=1, total=   1.0s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    8.6s finished


0.945814977974 {'alpha': 1}


In [59]:
#Just throw everything at a more robust model.
X = np.asarray(pd.concat([word_counts[filtered_words], pd.get_dummies(first_deps), pos_counts[pos_densities]], axis=1))

In [63]:
X_train, X_test, y_train, y_test = train_test_split(np.asarray(X), 
                                                    np.asarray(Y),
                                                    test_size=0.4,
                                                    random_state=0)

Y_train = np.where(y_train=='Austen', 1, 0)
Y_test = np.where(y_test=='Austen', 1, 0)

dtrain = xgb.DMatrix(X_train, label=Y_train)
dtest = xgb.DMatrix(X_test, label=Y_test)

param = {
    'max_depth': 5, 
    'eta': .2,
    'silent': 0, 
    'objective': 
    'binary:logistic', 
    'scale_pos_weight': (class_imb),  #Tell model about class imbalance
    'nthread':4,
    'eval_metric':'error'
}

plst = param.items()
num_round = 500
evallist = [(dtest, 'eval'), (dtrain, 'train')]
bst = xgb.train(plst, dtrain, num_round, evallist, verbose_eval=True) #We can watch outputs to see model slowly optimize.
#Breaks 90% accuracy at around 410 iterations, with vanishing returns around 500.

Y_ = np.where(bst.predict(dtrain)>.5, 1, 0)
tb = pd.crosstab(Y_, Y_train)
print(tb)
print((tb.iloc[0, 0]+tb.iloc[1, 1])/tb.sum().sum())

Y_ = np.where(bst.predict(dtest)>.5, 1, 0)
tb = pd.crosstab(Y_, Y_test)
print(tb)
print((tb.iloc[0, 0]+tb.iloc[1, 1])/tb.sum().sum())

[0]	eval-error:0.131057	train-error:0.123715
[1]	eval-error:0.129956	train-error:0.121512
[2]	eval-error:0.132159	train-error:0.129222
[3]	eval-error:0.129405	train-error:0.122247
[4]	eval-error:0.127203	train-error:0.117841
[5]	eval-error:0.127203	train-error:0.120411
[6]	eval-error:0.128855	train-error:0.118576
[7]	eval-error:0.126101	train-error:0.116006
[8]	eval-error:0.125	train-error:0.114537
[9]	eval-error:0.124449	train-error:0.113803
[10]	eval-error:0.123899	train-error:0.113436
[11]	eval-error:0.122797	train-error:0.111233
[12]	eval-error:0.119493	train-error:0.108664
[13]	eval-error:0.118943	train-error:0.108297
[14]	eval-error:0.11674	train-error:0.106094
[15]	eval-error:0.114537	train-error:0.102423
[16]	eval-error:0.114537	train-error:0.10022
[17]	eval-error:0.112885	train-error:0.098752
[18]	eval-error:0.112885	train-error:0.098018
[19]	eval-error:0.111784	train-error:0.096916
[20]	eval-error:0.110683	train-error:0.094714
[21]	eval-error:0.109582	train-error:0.093612
[22

[178]	eval-error:0.088656	train-error:0.052129
[179]	eval-error:0.088656	train-error:0.051762
[180]	eval-error:0.089207	train-error:0.051028
[181]	eval-error:0.089207	train-error:0.051028
[182]	eval-error:0.089207	train-error:0.050661
[183]	eval-error:0.088656	train-error:0.051028
[184]	eval-error:0.088656	train-error:0.051028
[185]	eval-error:0.088106	train-error:0.050294
[186]	eval-error:0.088656	train-error:0.049927
[187]	eval-error:0.088656	train-error:0.049927
[188]	eval-error:0.088656	train-error:0.049559
[189]	eval-error:0.089758	train-error:0.049192
[190]	eval-error:0.089758	train-error:0.049192
[191]	eval-error:0.089207	train-error:0.048825
[192]	eval-error:0.087555	train-error:0.047357
[193]	eval-error:0.087555	train-error:0.047724
[194]	eval-error:0.087555	train-error:0.047357
[195]	eval-error:0.087004	train-error:0.04699
[196]	eval-error:0.087004	train-error:0.04699
[197]	eval-error:0.087004	train-error:0.047357
[198]	eval-error:0.087004	train-error:0.04699
[199]	eval-error

KeyboardInterrupt: 

In [68]:
tkn = word_counts.iloc[123].text_sentence[10]

In [204]:
from sklearn.feature_extraction.text import CountVectorizer

In [205]:
vctr = CountVectorizer(
    ngram_range = (1, 3),
    stop_words = 'english',
    min_df = 3
)
lem_sents = []
for sent in word_counts.text_sentence:
    lem_sents.append([token.lemma_ for token in sent])

print(lem_sents[0])

['alice', 'be', 'begin', 'to', 'get', 'very', 'tired', 'of', 'sit', 'by', '-PRON-', 'sister', 'on', 'the', 'bank', ',', 'and', 'of', 'have', 'nothing', 'to', 'do', ':', 'once', 'or', 'twice', '-PRON-', 'have', 'peep', 'into', 'the', 'book', '-PRON-', 'sister', 'be', 'read', ',', 'but', '-PRON-', 'have', 'no', 'picture', 'or', 'conversation', 'in', '-PRON-', ',', "'", 'and', 'what', 'be', 'the', 'use', 'of', 'a', 'book', ',', "'", 'think', 'alice', "'", 'without', 'picture', 'or', 'conversation', '?', "'"]


In [206]:
recombined = [' '.join(x) for x in lem_sents]
word_counts['recombined_'] = [' '.join(x) for x in lem_sents]

vec_mat_tot = vctr.fit_transform(word_counts.recombined_)
vec_mat_aus = vctr.transform(word_counts[word_counts.text_source == 'Austen'].recombined_)
vec_mat_car = vctr.transform(word_counts[word_counts.text_source == 'Carroll'].recombined_)

In [207]:
tot_vec = np.asarray(vec_mat_tot.todense().sum(axis=0))[0]
aus_vec = np.asarray(vec_mat_aus.todense().sum(axis=0))[0]
car_vec = np.asarray(vec_mat_car.todense().sum(axis=0))[0]

In [208]:
aus_vec_part = aus_vec/tot_vec
car_vec_part = car_vec/tot_vec

In [209]:
min_parts = np.array([aus_vec_part, car_vec_part])
min_parts = min_parts.min(axis=0)

In [215]:
vocab = pd.Series(index=range(0, len(vctr.vocabulary_)))
for item in vctr.vocabulary_.items():
    vocab[item[1]] = item[0]

cutoff = .5
new_vocab = np.where(min_parts < cutoff, vocab, '')
new_vocab = list(set(new_vocab))
new_vocab.remove('')
len(new_vocab)

3008

In [220]:
new_vec = CountVectorizer(vocabulary=new_vocab)
new_vec.fit([])
X = new_vec.transform(word_counts.recombined_)
mod = BernoulliNB()
params = {
    'alpha': [.2, .6, .4, .5]
}

srch = GridSearchCV(mod, params, cv=10, verbose=1, n_jobs=2)
srch.fit(X, Y)
print(srch.best_score_, srch.best_params_)

Fitting 10 folds for each of 4 candidates, totalling 40 fits
0.930616740088 {'alpha': 0.5}


[Parallel(n_jobs=2)]: Done  40 out of  40 | elapsed:    0.7s finished


In [227]:
from sklearn.svm import SVC
svc = SVC(class_weight='balanced')
params = {
    'C': [.1, 1, 10],
    'gamma': [.1, 1]
}

srch = GridSearchCV(svc, params, cv=10, verbose=2, n_jobs=2)
srch.fit(X, Y)
print(srch.best_score_, srch.best_params_)

Fitting 10 folds for each of 6 candidates, totalling 60 fits
[CV] C=0.1, gamma=0.1 ................................................
[CV] C=0.1, gamma=0.1 ................................................
[CV] ................................. C=0.1, gamma=0.1, total=   3.1s
[CV] ................................. C=0.1, gamma=0.1, total=   3.1s
[CV] C=0.1, gamma=0.1 ................................................
[CV] C=0.1, gamma=0.1 ................................................
[CV] ................................. C=0.1, gamma=0.1, total=   2.0s
[CV] ................................. C=0.1, gamma=0.1, total=   2.0s
[CV] C=0.1, gamma=0.1 ................................................
[CV] C=0.1, gamma=0.1 ................................................
[CV] ................................. C=0.1, gamma=0.1, total=   2.2s
[CV] C=0.1, gamma=0.1 ................................................
[CV] ................................. C=0.1, gamma=0.1, total=   2.2s
[CV] C=0.1, gamm

[Parallel(n_jobs=2)]: Done  37 tasks      | elapsed:  1.2min


[CV] ..................................... C=1, gamma=1, total=   2.8s
[CV] C=1, gamma=1 ....................................................
[CV] ..................................... C=1, gamma=1, total=   2.6s
[CV] C=10, gamma=0.1 .................................................
[CV] ..................................... C=1, gamma=1, total=   2.5s
[CV] C=10, gamma=0.1 .................................................
[CV] .................................. C=10, gamma=0.1, total=   1.2s
[CV] C=10, gamma=0.1 .................................................
[CV] .................................. C=10, gamma=0.1, total=   1.1s
[CV] C=10, gamma=0.1 .................................................
[CV] .................................. C=10, gamma=0.1, total=   1.2s
[CV] C=10, gamma=0.1 .................................................
[CV] .................................. C=10, gamma=0.1, total=   1.2s
[CV] C=10, gamma=0.1 .................................................
[CV] .

[Parallel(n_jobs=2)]: Done  60 out of  60 | elapsed:  1.9min finished


0.895154185022 {'C': 1, 'gamma': 0.1}
